# Single image processing

In [1]:
import ee
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfgeXvvPkY25WLmjJEs7u4F7gIrakar-VHYu04d_uJPYWRodqmS6z1eIbEk



Successfully saved authorization token.


In [7]:
# Define area of interest
# If you have a GeoJSON file, copy paste.
# If you have a KML, export to GeoJSON (plenty of free tools online)
# or retrieve P

geoJSON = {
    "type": "FeatureCollection",
    "name": "Budrio_half-right",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": [
        { "type": "Feature", "properties": { "Name": "Budrio_campo_right", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.533083525108189, 44.570697786985733 ], [ 11.532456431504921, 44.569896556005418 ], [ 11.53276321111148, 44.569791821233608 ], [ 11.53338591526418, 44.570601794304793 ], [ 11.533083525108189, 44.570697786985733 ] ] ] } },
        { "type": "Feature", "properties": { "Name": "Budrio_campo_safe_half", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.53262979564736, 44.570842547510622 ], [ 11.532328100248961, 44.570445732016537 ], [ 11.53264162483709, 44.570339694294631 ], [ 11.532950828277439, 44.570738040751841 ], [ 11.53262979564736, 44.570842547510622 ] ] ] } }
    ]
}
coords = [geoJSON['features'][i]['geometry']['coordinates'] for i in range(2)]
aoi = ee.Geometry.MultiPolygon(coords)

# Filters definition
fil_orb95   = ee.Filter.eq('relativeOrbitNumber_start', 95)
fil_orb168  = ee.Filter.eq('relativeOrbitNumber_start', 168)
fil_orb117 = ee.Filter.eq('relativeOrbitNumber_start', 117)

sp17 = ee.Filter.date('2017-04-04', '2017-05-22')
su17 = ee.Filter.date('2017-05-22', '2017-09-15')
au17 = ee.Filter.date('2017-09-15', '2017-11-02')
tot17 = ee.Filter.date('2017-04-01', '2017-11-10')

# Get collection of images and filter
img = (ee.ImageCollection('COPERNICUS/S1_GRD')
        .filterBounds(aoi)
        .filter(tot17)
        .sort('system:time_start'))
        # .select('VV')

# acq_times = img.aggregate_array('system:time_start').getInfo()
# len([time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times])

# Extract data
geometry_title = input('Please provide a title for AoI geometry. (Default: Budrio_half-right)')
if not geometry_title: geometry_title='Budrio_half-right'

def extract_data(image:ee.Image):
    """Ausiliary function to extract data from an Image
    
    This function extracts spatial means and std.dev
    via spatial reducers (reduceRegion).
    Optimal implementation is to map this function
    on a whole ImageCollection via .map() and insert the
    return into a ee.FeatureCollection.
    
    Return
    ------
    ee.Feature
    
    """
    
    mean = image.reduceRegion(**{ 
        'reducer': ee.Reducer.mean(),
        'geometry': aoi,
    })
    
    dev = image.reduceRegion(**{ 
        'reducer': ee.Reducer.stdDev(),
        'geometry': aoi,
    })
        
    properties = {
        'Date': image.get('system:time_start'), # only way to get a timestr is an external operation
        'Geometry': geometry_title,
        'VV[dB]': mean.get('VV'),
        'VH[dB]': mean.get('VH'),
        'Angle[°]': mean.get('angle'),
        'VV_dev[dB]': dev.get('VV'),
        'VH_dev[dB]': dev.get('VH'),
        'Orb': image.get('relativeOrbitNumber_start'),
        'Pass': image.get('orbitProperties_pass'),
    }
    return ee.Feature(None, properties)

data = ee.FeatureCollection(img.map(extract_data))

data_out = data.getInfo()
# data_out.keys()
# data_out.get('features')[0].get('properties')
data_out_to_df = [e.get('properties') for e in data_out.get('features')]
df = pd.DataFrame.from_dict(data_out_to_df)

def clean_date(date:int):
    return time.strftime('%x %H', time.localtime((date)/1000))

df.Date = df.Date.apply(lambda x : pd.to_datetime(clean_date(x)))
df

Please provide a title for AoI geometry. (Default: Budrio_half-right) 


,Angle[°],Date,Geometry,Orb,Pass,VH[dB],VH_dev[dB],VV[dB],VV_dev[dB]
0,31.383728,2017-04-04 07:00:00,Budrio_half-right,168,DESCENDING,-20.830018,1.904476,-11.914824,1.816203
1,41.390998,2017-04-05 07:00:00,Budrio_half-right,95,DESCENDING,-22.619971,2.632017,-10.937961,2.027644
2,37.171040,2017-04-06 19:00:00,Budrio_half-right,117,ASCENDING,-22.724337,2.135879,-12.247353,1.337440
3,31.418842,2017-04-10 07:00:00,Budrio_half-right,168,DESCENDING,-22.204781,2.111794,-12.161258,1.736876
4,41.407985,2017-04-11 07:00:00,Budrio_half-right,95,DESCENDING,-23.842131,2.747528,-12.833449,1.617987
...,...,...,...,...,...,...,...,...,...
105,41.391327,2017-11-01 06:00:00,Budrio_half-right,95,DESCENDING,-19.798864,2.154767,-10.685643,1.487615
106,37.194078,2017-11-02 18:00:00,Budrio_half-right,117,ASCENDING,-20.128024,2.127804,-12.085922,1.565451
107,31.240426,2017-11-06 06:00:00,Budrio_half-right,168,DESCENDING,-14.985925,2.432004,-6.177465,1.872144
108,41.378068,2017-11-07 06:00:00,Budrio_half-right,95,DESCENDING,-14.897606,1.549332,-6.385496,1.586718


# Merging with in-situ data

Ref: Building_Golden.ipynb

In [8]:
# Platinum_Budrio has 2 sheets, one for 2017, one for 2020

import datetime as dtt

database = pd.ExcelFile('Platinum_Budrio.xlsx', engine='openpyxl')

data_1h_2017 = database.parse('2017_1h')
data_1h_2017['Ora'] = pd.to_datetime(data_1h_2017['Ora'].astype('str')).apply(lambda x: x.time())
data_1h_2017['BiOra'] = data_1h_2017['Ora'].apply(lambda x: 2*np.around(x.hour/2))
data_1h_2017['Data'] = pd.to_datetime(data_1h_2017['Data'].astype('str')).apply(lambda x: x.date())
data_1h_2017['Mese'] = data_1h_2017['Data'].apply(lambda x: x.month)
data_1h_2017['Date'] = data_1h_2017.apply(lambda r : dtt.datetime.combine(r['Data'],r['Ora']),1)
data_1h_2017 = data_1h_2017.drop(['ID', 'Data', 'Ora', '214Pb[cps]', 'BiOra', 'Mese'],axis=1)
data_1h_2017

,SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C],Date
0,0.173254,0.0,0.0,17.4097,2017-04-03 11:00:00
1,0.174514,0.0,0.0,19.1982,2017-04-03 12:00:00
2,0.178998,0.0,0.0,20.9032,2017-04-03 13:00:00
3,0.206764,0.0,0.0,21.8341,2017-04-03 14:00:00
4,0.169945,0.0,0.0,22.3737,2017-04-03 15:00:00
...,...,...,...,...,...
5120,0.139442,0.0,0.0,11.3791,2017-11-02 19:00:00
5121,0.135318,0.0,0.0,10.9391,2017-11-02 20:00:00
5122,0.149532,0.0,0.0,10.7549,2017-11-02 21:00:00
5123,0.130251,0.0,0.0,11.1445,2017-11-02 22:00:00


In [16]:
golden = pd.merge(left=df, right=data_1h_2017, how='left', on='Date').iloc[:-3]; golden

,Angle[°],Date,Geometry,Orb,Pass,VH[dB],VH_dev[dB],VV[dB],VV_dev[dB],SWC[m3/m3],Pioggia[mm],Irrigazione[mm],Temperatura[°C]
0,31.383728,2017-04-04 07:00:00,Budrio_half-right,168,DESCENDING,-20.830018,1.904476,-11.914824,1.816203,0.170173,0.0,0.0,8.7339
1,41.390998,2017-04-05 07:00:00,Budrio_half-right,95,DESCENDING,-22.619971,2.632017,-10.937961,2.027644,0.147105,0.0,0.0,11.8548
2,37.171040,2017-04-06 19:00:00,Budrio_half-right,117,ASCENDING,-22.724337,2.135879,-12.247353,1.337440,0.150527,0.0,0.0,15.6591
3,31.418842,2017-04-10 07:00:00,Budrio_half-right,168,DESCENDING,-22.204781,2.111794,-12.161258,1.736876,0.159550,0.0,0.0,8.5507
4,41.407985,2017-04-11 07:00:00,Budrio_half-right,95,DESCENDING,-23.842131,2.747528,-12.833449,1.617987,0.160673,0.0,0.0,10.5575
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,41.380256,2017-10-26 07:00:00,Budrio_half-right,95,DESCENDING,-19.882918,1.802317,-10.557746,1.581814,0.155637,0.0,0.0,5.1388
103,37.172670,2017-10-27 19:00:00,Budrio_half-right,117,ASCENDING,-19.356794,1.974589,-11.910805,1.812534,0.155525,0.0,0.0,14.4822
104,31.274119,2017-10-31 06:00:00,Budrio_half-right,168,DESCENDING,-20.319962,2.037577,-8.338587,1.739120,0.167105,0.0,0.0,4.9978
105,41.391327,2017-11-01 06:00:00,Budrio_half-right,95,DESCENDING,-19.798864,2.154767,-10.685643,1.487615,0.124052,NaN,0.0,NaN


In [17]:
save = input("Wanna save in root directory? [y/n] ")
if save=='y': golden.to_csv(f'Golden_GEE.csv', sep = '\t')

Wanna save in root directory? [y/n]  y
